In [2]:
import pandas as pd

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
def estimation(data):
    if data <= 2:
        return 'Низкий рейтинг'
    elif data >=4.5:
        return 'Высокий рейтинг'
    else:
        return 'Средний рейтинг'
    

data1 = pd.read_csv(r'C:\Users\user\Desktop\PYDP-19-DSU-4\NumPy, pandas, MPL\10. Pandas\ml-latest-small\ratings.csv')
data1 = pd.DataFrame(data1.groupby(by = 'movieId').rating.mean())
data1['class'] = data1['rating'].apply(estimation)
data1




,rating,class
movieId,,
1,3.872470,Средний рейтинг
2,3.401869,Средний рейтинг
3,3.161017,Средний рейтинг
4,2.384615,Средний рейтинг
5,3.267857,Средний рейтинг
...,...,...
161944,5.000000,Высокий рейтинг
162376,4.500000,Высокий рейтинг
162542,5.000000,Высокий рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
1 'Центр': ['москва', 'тула', 'ярославль']  
2 'Северо-Запад': ['петербург', 'псков', 'мурманск']  
3 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
}

Результат классификации запишите в отдельный столбец region.

In [383]:
# Я решила усложнить задачу путем скачивания информации об отношении городов к округам с определенного сайта по ссылке,
# указанной ниже: 'https://superresearch.ru/federal-okrug'

data2 = pd.read_csv(r"C:\Users\user\Desktop\PYDP-19-DSU-4\NumPy, pandas, MPL\10. Pandas\ml-latest-small\keywords.csv") #получение информации из файла
        
    # получение информации об отношении городов к округам с стороннего сайта, выбрав только необходимые столбцы и убрав ненужные строки
data3 = pd.read_html('https://superresearch.ru/federal-okrug')[3].iloc[1:,[1,3]] 
        
    # назвала столбцы таблицы
data3.columns = ['town', 'district']
        
    # удалила строки в таблице, где название округа ('district') не указано, а вместого этого идет обозначение пустой строки NaN
data3 = data3[data3.district.notnull()]
        
    # транспонировала таблицу и столбцец 'district' указала в качестве названий столбцов, а города- значения строк.
    # перевела DF в формат словаря в переменную geo_data, где ключами являются название округов, а значениями - списки городов.
geo_data = data3.pivot(columns = 'district', values = 'town').to_dict('list')
        
    # в цикле убрала из списков городов все значения пустых строк NaN
for i in geo_data:   
    geo_data[i] = [x.lower() for x in geo_data[i] if str(x) != 'nan']  #geo_data[i] = list(set(geo_data[i])) - к сожалению этот способ всегда 1 значение Nan оставит

        
    #функция, проверяющая наличие название города в тексте
def geo(keyword):
    for reg,tow in geo_data.items():
        for i in tow:            
            if i in keyword.lower().split():
                return reg
    return 'undefined'
    
data2['district'] = data2['keyword'].apply(geo)

print(data2) #можно посмотреть выгрузку всей таблицы,но это не совсем информативно, т.к. таблица состоит из 100 тысяч строк

    # поэтому сделала сводную статистику по регионам и кол-ву найденных строк с городами относящимся к ним 
print(data2.groupby('district').count())




                       keyword     shows   district
0                           вк  64292779  undefined
1                одноклассники  63810309  undefined
2                        порно  41747114  undefined
3                         ютуб  39995567  undefined
4                    вконтакте  21014195  undefined
...                        ...       ...        ...
99995   эльдорадо старый оскол      3705  undefined
99996      frigate для firefox      3630  undefined
99997                   укрсиб      3630  undefined
99998  погода в ялте на неделю      3688  undefined
99999                     ггму      3630  undefined

[100000 rows x 3 columns]
                   keyword  shows
district                         
undefined            96419  96419
Дальневосточный        114    114
Приволжский            860    860
Северо-Западный        148    148
Северо-Кавказский       49     49
Сибирский              661    661
Уральский              444    444
Центральный           1041   1041
Южный    

### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [421]:
title_mov = pd.read_csv(r"C:\Users\user\Desktop\PYDP-19-DSU-4\PYDP19_DSU4_homeworks\Python part 2 (NumPy, pandas, MPL)\Pandas\Movies_set\movies.csv").iloc[:, 0:2]
rating_mov = pd.read_csv(r"C:\Users\user\Desktop\PYDP-19-DSU-4\PYDP19_DSU4_homeworks\Python part 2 (NumPy, pandas, MPL)\Pandas\Movies_set\ratings.csv").iloc[:, 1:3]
years = list(range(1950,2020))

def production_year(name):
    for i in years:
        if str(i) in name:
            return i
    return int(1900)

title_mov['year'] = title_mov['title'].apply(production_year)
title_mov = title_mov.set_index('movieId').join(rating_mov.set_index('movieId'))
title_mov.groupby('year').mean('rating').sort_values(by = ['rating'], ascending = False).head(50)



        
    




,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
1964,3.940160
1974,3.935622
1967,3.922572
1975,3.879121
